#Load Packages

In [1]:
import pandas as pd
import json
from datetime import datetime
import numpy as np
from ast import literal_eval

In [2]:
receipts = pd.read_json('receipts.json',lines=True)
users = pd.read_json('users.json',lines=True)
brands = pd.read_json('brands.json',lines=True)

# Parse Receipt Data

In [3]:
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [4]:
receipts['_id'] = receipts['_id'].apply(lambda x: x['$oid'])

In [5]:
def date_converter(x):
  try:
    return(datetime.utcfromtimestamp(int(x['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
  except TypeError:
    return(np.nan)

In [6]:
receipts['createDate'] = receipts['createDate'].apply(lambda x: date_converter(x))
receipts['dateScanned'] = receipts['dateScanned'].apply(lambda x: date_converter(x))
receipts['finishedDate'] = receipts['finishedDate'].apply(lambda x: date_converter(x))
receipts['modifyDate'] = receipts['modifyDate'].apply(lambda x: date_converter(x))
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].apply(lambda x: date_converter(x))
receipts['purchaseDate'] = receipts['purchaseDate'].apply(lambda x: date_converter(x))

In [7]:
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaN,2021-01-03 15:25:42,NaN,5.0,2021-01-03 00:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [8]:
rewardsReceiptItemList = receipts[['_id','rewardsReceiptItemList']]
receipts.drop(columns = 'rewardsReceiptItemList', inplace = True)

In [9]:
rewardsReceiptItemList = rewardsReceiptItemList.explode('rewardsReceiptItemList')
rewardsReceiptItemList.reset_index(inplace=True)

In [10]:
rewardsReceiptItemList.rewardsReceiptItemList = rewardsReceiptItemList.rewardsReceiptItemList.fillna('{}')

In [11]:
rewardsReceiptItemList.rewardsReceiptItemList = rewardsReceiptItemList.rewardsReceiptItemList.apply(lambda x:str(x))
rewardsReceiptItemList.rewardsReceiptItemList = rewardsReceiptItemList.rewardsReceiptItemList.apply(literal_eval)

In [12]:
rewardsReceiptItemList = rewardsReceiptItemList.join(pd.json_normalize(rewardsReceiptItemList.pop('rewardsReceiptItemList')))

In [13]:
rewardsReceiptItemList.head()

,index,_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptStatus     1119 non-null   object 
 12  totalSpent               684 non-null    float64
 13  userId                   1119 non-null   object 
dtypes: float64(4), object(10

In [15]:
rewardsReceiptItemList.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7381 entries, 0 to 7380
Data columns (total 36 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   index                               7381 non-null   int64  
 1   _id                                 7381 non-null   object 
 2   barcode                             3090 non-null   object 
 3   description                         6560 non-null   object 
 4   finalPrice                          6767 non-null   object 
 5   itemPrice                           6767 non-null   object 
 6   needsFetchReview                    813 non-null    object 
 7   partnerItemId                       6941 non-null   object 
 8   preventTargetGapPoints              358 non-null    object 
 9   quantityPurchased                   6767 non-null   float64
 10  userFlaggedBarcode                  337 non-null    object 
 11  userFlaggedNewItem                  323 non

# Parse User Data

In [16]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [17]:
users['_id'] = users['_id'].apply(lambda x: x['$oid'])
users['createdDate'] = users['createdDate'].apply(lambda x: date_converter(x))
users['lastLogin'] = users['lastLogin'].apply(lambda x: date_converter(x))

In [18]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


In [19]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


# Parse Brand Data

In [20]:
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [21]:
brands['_id'] = brands['_id'].apply(lambda x: x['$oid'])

In [22]:
cpg = brands[['_id','cpg']]
brands.drop(columns = 'cpg', inplace = True)

In [23]:
cpg = cpg.join(pd.json_normalize(cpg.pop('cpg')))

In [24]:
cpg

,_id,$ref,$id.$oid
0,601ac115be37ce2ead437551,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...,...
1162,5f77274dbe37ce6b592e90c0,Cogs,5f77274dbe37ce6b592e90bf
1163,5dc1fca91dda2c0ad7da64ae,Cogs,53e10d6368abd3c7065097cc
1164,5f494c6e04db711dd8fe87e7,Cogs,5332fa12e4b03c9a25efd1e7
1165,5a021611e4b00efe02b02a57,Cogs,5332f5f6e4b03c9a25efd0b4


In [25]:
cpg.rename(columns={"$ref": "ref", "$id.$oid": "oid"}, inplace = True)

In [26]:
cpg

,_id,ref,oid
0,601ac115be37ce2ead437551,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...,...
1162,5f77274dbe37ce6b592e90c0,Cogs,5f77274dbe37ce6b592e90bf
1163,5dc1fca91dda2c0ad7da64ae,Cogs,53e10d6368abd3c7065097cc
1164,5f494c6e04db711dd8fe87e7,Cogs,5332fa12e4b03c9a25efd1e7
1165,5a021611e4b00efe02b02a57,Cogs,5332f5f6e4b03c9a25efd0b4


In [27]:
brands.topBrand.unique()

array([ 0., nan,  1.])

In [28]:
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   _id           1167 non-null   object 
 1   barcode       1167 non-null   int64  
 2   category      1012 non-null   object 
 3   categoryCode  517 non-null    object 
 4   name          1167 non-null   object 
 5   topBrand      555 non-null    float64
 6   brandCode     933 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 63.9+ KB


In [29]:
cpg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   _id     1167 non-null   object
 1   ref     1167 non-null   object
 2   oid     1167 non-null   object
dtypes: object(3)
memory usage: 27.5+ KB


# SQL

In [30]:
pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
import pandasql as ps

In [39]:
result = ps.sqldf(
    " select average_spend_Accepted, average_spend_Rejected, total_number_items_Accepted, total_number_items_Rejected from (select sum(totalSpent)/count(_id) as average_spend_Accepted, sum(purchasedItemCount) as total_number_items_Accepted from receipts where rewardsReceiptStatus == 'FINISHED' or rewardsReceiptStatus == 'FLAGGED') as a cross join (select sum(totalSpent)/count(_id) as average_spend_Rejected, sum(purchasedItemCount) as total_number_items_Rejected from receipts where rewardsReceiptStatus == 'REJECTED') as b"
)


In [40]:
result

,average_spend_Accepted,average_spend_Rejected,total_number_items_Accepted,total_number_items_Rejected
0,88.9775,23.326056,9198.0,173.0


# Data Quality Issues Evaluation

Investigation of null values in lastLogin column from users table

In [49]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


In [52]:
consumer = users.loc[users['role'] == 'consumer']

In [64]:
consumer.nunique()

_id             204
active            2
createdDate     204
lastLogin       163
role              1
signUpSource      2
state             7
dtype: int64

Among 204 unique consumer ids, there are 41 entries do not have the last login record. These NaNs indicate that these consumers might become inactive. 

After going through Fetch Reward phone application's user journey, I think failing the receipt validation process might be a user pain point that causes users to churn and leave null values in users table.

Step1 1: Select and store NaN lastLogin userIDs from users table into a list

In [55]:
users_churn = consumer.loc[consumer['lastLogin'].isna()]

In [56]:
users_churn

,_id,active,createdDate,lastLogin,role,signUpSource,state
97,5ff616a68f142f11dd189163,True,2021-01-06 19:59:34,NaN,consumer,Email,KY
143,5ffe115404929101d0aaebb2,True,2021-01-12 21:15:00,NaN,consumer,Email,AL
148,5ffe115404929101d0aaebb2,True,2021-01-12 21:15:00,NaN,consumer,Email,AL
170,5e27526d0bdb6a138c32b556,True,2020-01-21 19:35:09,NaN,consumer,Google,WI
180,6002475cfb296c121a81b98d,True,2021-01-16 01:54:36,NaN,consumer,Email,WI
...,...,...,...,...,...,...,...
381,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03,NaN,consumer,Email,NaN
382,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03,NaN,consumer,Email,NaN
389,60217fa799409b11fcf899fe,True,2021-02-08 18:15:03,NaN,consumer,Email,WI
420,5fb0a078be5fc9775c1f3945,True,2020-11-15 03:28:56,NaN,consumer,Google,AL


In [ ]:
with pd.option_context("display.max_rows", 1000):
    display(users_churn)

In [58]:
churn_lst = users_churn['_id'].tolist()

Step 2: Use [churn_lst] to locate [_id] in receipts table

In [59]:
churn_receipt = receipts.loc[receipts['userId'].isin(churn_lst)]

Step 3: Filter out accepted rewards

In [60]:
churn_receipt_rejected = churn_receipt.loc[(churn_receipt['rewardsReceiptStatus'] != 'FINISHED') & (churn_receipt['rewardsReceiptStatus'] != 'FLAGGED')]

In [61]:
churn_receipt_rejected

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
246,5ffe17e20a720f05ac0000e7,750.0,"Receipt number 1 completed, bonus point schedu...",2021-01-12 21:42:58,2021-01-12 21:42:58,NaN,2021-01-13 16:00:54,NaN,750.0,2021-01-12 00:00:00,1.0,REJECTED,2.29,5ffe115404929101d0aaebb2
300,600205f60a7214ad4c00007b,NaN,NaN,2021-01-15 21:15:32,2021-01-15 21:15:32,NaN,2021-01-15 21:15:32,NaN,NaN,NaN,NaN,SUBMITTED,NaN,6000b75bbe5fc96dfee1d4d3
302,6001bb110a720f05f3000085,NaN,NaN,2021-01-15 15:56:01,2021-01-15 15:56:01,NaN,2021-01-15 15:56:01,NaN,NaN,NaN,NaN,SUBMITTED,NaN,6000b75bbe5fc96dfee1d4d3
307,600247cd0a7214ad4c00007e,NaN,NaN,2021-01-16 01:56:29,2021-01-16 01:56:29,NaN,2021-01-16 01:56:29,NaN,NaN,NaN,NaN,SUBMITTED,NaN,6002475cfb296c121a81b98d
308,60025eb70a720f05f300008e,NaN,NaN,2021-01-16 03:34:15,2021-01-16 03:34:15,NaN,2021-01-16 03:34:15,NaN,NaN,NaN,NaN,SUBMITTED,NaN,60025c65fb296c4ef805d9e6
311,600254550a720f05f300008c,NaN,NaN,2021-01-16 02:49:57,2021-01-16 02:49:57,NaN,2021-01-16 02:49:57,NaN,NaN,NaN,NaN,SUBMITTED,NaN,6002541ae257124ec6b99a3a
312,60024fa00a7214ad4c00007f,NaN,NaN,2021-01-16 02:29:52,2021-01-16 02:29:52,NaN,2021-01-16 02:29:52,NaN,NaN,NaN,NaN,SUBMITTED,NaN,60024f24e257124ec6b99a13
313,60025f550a7214ad4c000084,NaN,NaN,2021-01-16 03:36:53,2021-01-16 03:36:53,NaN,2021-01-16 03:36:53,NaN,NaN,NaN,NaN,SUBMITTED,NaN,60025c65fb296c4ef805d9e6
319,600498e60a720f05f3000093,NaN,NaN,2021-01-17 20:07:02,2021-01-17 20:07:02,NaN,2021-01-17 20:07:02,NaN,NaN,NaN,NaN,SUBMITTED,NaN,600498c4e257124ec6b9a331
322,600496350a7214ad4c000087,NaN,NaN,2021-01-17 19:55:33,2021-01-17 19:55:33,NaN,2021-01-17 19:55:33,NaN,NaN,NaN,NaN,SUBMITTED,NaN,600495c3fb296c4ef805e1fc


In [62]:
churn_rejected_id = churn_receipt_rejected['_id'].tolist()

In [63]:
rewardsReceiptItemList.loc[rewardsReceiptItemList['_id'].isin(churn_rejected_id)] 

,index,_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
569,246,5ffe17e20a720f05ac0000e7,072785048119,Germ-X Hand Sanitizer - Original,2.29,2.29,NaN,0,NaN,1.0,...,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,2.29,NaN
716,300,600205f60a7214ad4c00007b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
800,302,6001bb110a720f05f3000085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1179,307,600247cd0a7214ad4c00007e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1180,308,60025eb70a720f05f300008e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1402,311,600254550a720f05f300008c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1403,312,60024fa00a7214ad4c00007f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1404,313,60025f550a7214ad4c000084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1927,319,600498e60a720f05f3000093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2114,322,600496350a7214ad4c000087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finding: This thread leads to a matrix of NaNs (with one outlier) in the rewardsReceiptItemList table. More evidence is needed to support that reward validation rejection is an actual cause of null values in users table. 